In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 37.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.14.0-dev20230620


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-06-20 13:25:11--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2023-06-20 13:25:11 (66.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-06-20 13:25:11--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('train-data.tsv',sep = '\t')

In [5]:
data.columns = ['class','message']

In [6]:
data.head()

,class,message
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [7]:
x_train = data['message']

In [8]:
x_train[:5]

0                             you can never do nothing
1    now u sound like manky scouse boy steve,like! ...
2    mum say we wan to go then go... then she can s...
3    never y lei... i v lazy... got wat? dat day ü ...
4    in xam hall boy asked girl tell me the startin...
Name: message, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize

In [12]:
lemmatizer =  WordNetLemmatizer()

def preprocess(text):

   filtered = " ".join([lemmatizer.lemmatize(word).lower() for word in text.split() if word.lower() not in stopwords.words('english')])

   return filtered




In [13]:
preprocess("Hello$ this is $clean & sentence")

'hello$ $clean & sentence'

In [14]:
x_train = data['message'].apply(lambda text: preprocess(text))
x_train[0:5]

0                                        never nothing
1    u sound like manky scouse boy steve,like! trav...
2    mum say wan go go... shun bian watch da glass ...
3    never lei... v lazy... got wat? dat day ü send...
4    xam hall boy asked girl tell starting term dis...
Name: message, dtype: object

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

max_word,max_len = 1000,400

In [16]:
t = Tokenizer(num_words=max_word)
t.fit_on_texts(x_train)

In [17]:
sequences = t.texts_to_sequences(x_train)
sequences[0:5]

[[205, 263],
 [1, 336, 15, 254, 15, 46, 323, 40, 416, 1, 7, 358, 454],
 [632, 63, 199, 10, 10, 255, 46],
 [205, 515, 182, 878, 20, 86, 235, 18, 31, 25, 46, 132, 75, 28],
 [254, 433, 200, 47, 795, 358, 382, 297, 157, 37, 144, 96, 987]]

In [18]:
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
sequences_matrix[:5]

array([[  0,   0,   0, ...,   0, 205, 263],
       [  0,   0,   0, ...,   7, 358, 454],
       [  0,   0,   0, ...,  10, 255,  46],
       [  0,   0,   0, ..., 132,  75,  28],
       [  0,   0,   0, ..., 144,  96, 987]], dtype=int32)

In [19]:
input = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_word, 50, input_length=max_len)(input)
x = tf.keras.layers.LSTM(64)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=input, outputs=x)
model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)


In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400)]             0         
                                                                 
 embedding (Embedding)       (None, 400, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 96337 (376.32 KB)
Trainable params: 96337 (376.

In [21]:
y_train = data['class'].apply(lambda x: 0 if x == 'ham' else 1)

In [22]:
y_train = np.array(y_train)

In [23]:
y_train[0:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [24]:
r = model.fit(sequences_matrix, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
27/27 [==============================] - 33s 1s/step - loss: 0.3613 - accuracy: 0.8719 - val_loss: 0.1982 - val_accuracy: 0.9043
Epoch 2/10
27/27 [==============================] - 20s 755ms/step - loss: 0.1089 - accuracy: 0.9698 - val_loss: 0.0838 - val_accuracy: 0.9856
Epoch 3/10
27/27 [==============================] - 22s 823ms/step - loss: 0.0684 - accuracy: 0.9880 - val_loss: 0.1142 - val_accuracy: 0.9880


In [25]:
def preprocessing(text):
  x = text.apply(lambda x: preprocess(x))
  x = t.texts_to_sequences(x)
  return sequence.pad_sequences(x, maxlen=max_len)

In [26]:
test_data = pd.read_csv('valid-data.tsv',sep = '\t')
test_data.columns = ['class','message']

In [27]:
test_data.head()

,class,message
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [28]:
x_test = test_data['message']
y_test = test_data['class'].apply(lambda x: 0 if x == 'ham' else 1)

In [29]:
evaluation = model.evaluate(preprocessing(x_test), y_test)

44/44 [==============================] - 4s 69ms/step - loss: 0.0827 - accuracy: 0.9842


In [30]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict(preprocessing(pd.Series([pred_text])))
  return (pred,"ham" if pred < 0.5 else "spam")

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 490ms/step
(array([[0.]], dtype=float32), 'ham')


In [31]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 37ms/step
(array([[0.]], dtype=float32), 'ham')
1/1 [==============================] - 0s 44ms/step
(array([[0.6826402]], dtype=float32), 'spam')
1/1 [==============================] - 0s 39ms/step
(array([[0.]], dtype=float32), 'ham')
1/1 [==============================] - 0s 35ms/step
(array([[1.1937343]], dtype=float32), 'spam')
1/1 [==============================] - 0s 36ms/step
(array([[1.5397125]], dtype=float32), 'spam')
1/1 [==============================] - 0s 47ms/step
(array([[0.]], dtype=float32), 'ham')
1/1 [==============================] - 0s 35ms/step
(array([[0.]], dtype=float32), 'ham')
You passed the challenge. Great job!
